In [2]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [3]:
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
import gensim.downloader as api
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Embedding, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
#load dataset
url = "https://raw.githubusercontent.com/ironhack-labs/project-nlp-challenge/refs/heads/main/dataset/data.csv"
df = pd.read_csv(url)

#drop unnecessary columns
df = df.drop(columns=['date','subject','title'],axis=1)

#fill missing values in the text column
df['text'] = df['text'].fillna("")


# Load Pre-trained Word2Vec Model



In [5]:
#load pre-trained Word2Vec model (Google News 300)
w2v_model = api.load("word2vec-google-news-300")
embedding_dim = 300  #word vector size


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [6]:
#function to convert text into a list of word vectors
def text_to_sequence(text, model):
    words = word_tokenize(text.lower()) #Tokenize text into words
    return [model[word] for word in words if word in model]  #extract available word vectors

In [8]:
#convert text into lists of word vectors
nltk.download('punkt_tab')
df['vectors'] = df['text'].apply(lambda x: text_to_sequence(str(x), w2v_model))


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


# Split Data into Training and Testing Sets

In [9]:
#set a fixed sentence length (Padding/Truncating)
max_length = 30

#prepare input data (X)
X = pad_sequences(df['vectors'], maxlen=max_length, dtype='float32', padding='post', truncating='post', value=0.0)

#prepare labels (y)
y = df['label'].values

#split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Build and Compile CNN Model


In [10]:
model = Sequential([
    Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(max_length, embedding_dim)),
    Dropout(0.5),
    GlobalMaxPooling1D(),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),  # إضافة L2 Regularization
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#add early stop
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Training



In [11]:
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test), callbacks=[early_stop])

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 61ms/step - accuracy: 0.8868 - loss: 0.5632 - val_accuracy: 0.9964 - val_loss: 0.1202
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 56ms/step - accuracy: 0.9963 - loss: 0.0802 - val_accuracy: 0.9965 - val_loss: 0.0535
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 56ms/step - accuracy: 0.9977 - loss: 0.0325 - val_accuracy: 0.9974 - val_loss: 0.0256
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 55ms/step - accuracy: 0.9981 - loss: 0.0226 - val_accuracy: 0.9975 - val_loss: 0.0204
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 55ms/step - accuracy: 0.9974 - loss: 0.0205 - val_accuracy: 0.9975 - val_loss: 0.0192
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 56ms/step - accuracy: 0.9984 - loss: 0.0165 - val_accuracy: 0.9977 - val_loss: 0.0185
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 56ms/step - accuracy: 0.9987 - loss: 0.0132 - val_accuracy: 0.9974 - val_loss: 0.0173
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 56ms/step - accuracy: 0.9987 - loss: 0.0136 - 

# Evaluation


In [12]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Improved CNN Model Accuracy: {accuracy:.4f}")


250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9981 - loss: 0.0123
Improved CNN Model Accuracy: 0.9975


#save the model



In [17]:
model.save("cnn_model.h5")


#Generate Predictions and Save to CSV


In [16]:
import pandas as pd

y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)  #convert labels into 0 and 1

predictions_df = pd.DataFrame({'predicted_label': y_pred.flatten()})

predictions_df.to_csv('cnn_predictions.csv', index=False)

print("predictions.csv is saved successfuly")


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
predictions.csv is saved successfuly
